In [2]:
from collections import Counter
from tqdm import tqdm
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
ann_files1 = pd.read_csv("../../data/cigro_ad_infos_form - thesmc/table1.csv")
ann_files2 = pd.read_csv("../../data/cigro_ad_infos_form - thesmc/table2.csv")
ann_files3 = pd.read_csv("../../data/cigro_ad_infos_form - thesmc/table3.csv")
ann_files4 = pd.read_csv("../../data/cigro_ad_infos_form - thesmc/table4.csv")
ann_files5 = pd.read_csv("../../data/cigro_ad_infos_form - thesmc/table5.csv")
ann_files6 = pd.read_csv("../../data/cigro_ad_infos_form - thesmc/table6.csv")


files = list(ann_files1["광고명"]) + list(ann_files2["광고명"]) + list(ann_files3["광고명"]) + list(ann_files4["광고명"]) +\
    list(ann_files5["광고명"]) + list(ann_files6["광고명"])
label = [0] * len(list(ann_files1["광고명"])) + [1] * len(list(ann_files2["광고명"])) + [2] * len(list(ann_files3["광고명"])) +\
    [3] * len(list(ann_files4["광고명"])) + [4] * len(list(ann_files5["광고명"])) + [5] * len(list(ann_files6["광고명"]))

In [3]:
skf = StratifiedKFold(n_splits = 5, random_state=42, shuffle=True)

folds = []
for idx, (train_idx, val_idx) in enumerate(skf.split(files, label)):
    folds.append((train_idx, val_idx))

In [4]:
train_idx, val_idx = folds[0]

In [5]:
np.array([files, label])[:, train_idx]

array([['굿티 프리미엄 구강세정기', '굿티 14일 무료체험 이벤트', '{keyword:구강건강}, 굿티로 시작!',
        ..., '단일_청정커튼_착시효과_문초희_211229', '단일_사계절이불_후기강조_문초희_211217_사본',
        '211021_매일세로수건_단일_호텔인테리어 - Copy'],
       ['0', '0', '0', ..., '5', '5', '5']], dtype='<U68')

In [6]:
train_ann_files = pd.DataFrame(np.array([files, label])[:, train_idx].transpose())
val_ann_files = pd.DataFrame(np.array([files, label])[:, val_idx].transpose())
print('train data length: ', len(train_ann_files))
print('val data length: ', len(val_ann_files))

train data length:  16451
val data length:  4113


In [7]:
ann_files = pd.DataFrame(np.array([files, label]).transpose())
ann_files.columns = ['text', 'label']

In [8]:
#ann_files.to_csv("data.csv")

In [9]:
train_ann_files.columns = ['text', 'label']
val_ann_files.columns = ['text', 'label']

In [10]:
#!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

In [11]:
import torch
import torch.utils.data as data
import numpy as np
from PIL import Image
from io import BytesIO
import requests

In [12]:
from transformers import ElectraModel, ElectraTokenizer
import torch.nn as nn
model = ElectraModel.from_pretrained("monologg/koelectra-base-v3-discriminator")  # KoELECTRA-Base-v3
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")


Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
class Dataset(data.Dataset):
    def __init__(self, args='', ann_files=''):
        super(Dataset, self).__init__()
        self.args = args
        self.ann_files = ann_files
        
    def __getitem__(self, index):
        ann_file = self.ann_files.loc[index]
        text = ann_file['text']
        input_ids, attention_masks = self.tokenize([text])
        inputs = (torch.as_tensor(input_ids, dtype=torch.long), torch.as_tensor(attention_masks, dtype=torch.long))

        target = torch.tensor([int(ann_file['label'])], dtype=torch.long)
                
        return inputs, target
    
    def __len__(self):
        return len(self.ann_files)


    def tokenize(self, sequences):
        input_ids = []
        attention_masks = []

        for seq in sequences:
            encoded_dict = tokenizer.encode_plus(
                seq,  # Sentence to encode.
                add_special_tokens=True,  # Add '[CLS]' and '[SEP]'
                max_length=32,  # Pad & truncate all sentences.
                truncation=True,
                padding='max_length',
                return_attention_mask=True,  # Construct attn. masks.
                return_tensors='pt',  # Return pytorch tensors.
            )

            input_ids.append(encoded_dict['input_ids'])
            attention_masks.append(encoded_dict['attention_mask'])

        input_ids = torch.cat(input_ids, dim=0)
        attention_masks = torch.cat(attention_masks, dim=0)

        return input_ids.squeeze(), attention_masks.squeeze()

In [15]:
from config import Config
args = Config()

In [16]:
dataset_train = Dataset(ann_files=train_ann_files)

In [20]:
dataset_train[0]

((tensor([    2,  2110,  4017,  9350, 21378,  4246,  4142,  4031,     3,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0]),
  tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0])),
 tensor([0]))

In [17]:
loader_train=data.DataLoader(dataset_train, batch_size = args.batch_size, shuffle=False)

In [18]:
class Model(nn.Module):


    def __init__(self, config):
        super(Model, self).__init__()
        self.model = ElectraModel.from_pretrained("monologg/koelectra-base-v3-discriminator") 
        self.dropout = nn.Dropout(p=config.drop_rate)
        self.fc = nn.Linear(config.feature_dim, config.num_classes)
        torch.nn.init.xavier_uniform_(self.fc.weight) 
        
    def forward(self, x):
        x = self.model(x[0], attention_mask=x[1],)[0][:,0,:]
        x = self.dropout(x)
        return self.fc(x)
    

In [95]:
model = Model(args)

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [20]:
[x.to('cpu') for x in dataset_train[0][0]]

[tensor([    2,  2110,  4017,  9350, 21378,  4246,  4142,  4031,     3,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0]),
 tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0])]

In [97]:
model(next(iter(loader_train)))

torch.Size([16, 2])

In [64]:
args.drop_rate

0.3

In [1]:
from config import Config
args = Config()
from train import Train
Train(args)

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


train data length:  16451
val data length:  4113


  0%|          | 0/1029 [00:00<?, ?it/s]

tensor(1.6865, device='cuda:0', grad_fn=<NllLossBackward0>)
16


  0%|          | 1/1029 [00:00<02:40,  6.39it/s]

...

100%|█████████▉| 1025/1029 [01:25<00:00, 11.31it/s]

tensor(0.1685, device='cuda:0', grad_fn=<NllLossBackward0>)
16
tensor(0.1588, device='cuda:0', grad_fn=<NllLossBackward0>)
16
tensor(0.3101, device='cuda:0', grad_fn=<NllLossBackward0>)
16


100%|██████████| 1029/1029 [01:25<00:00, 12.05it/s]


tensor(0.1819, device='cuda:0', grad_fn=<NllLossBackward0>)
16
tensor(0.5092, device='cuda:0', grad_fn=<NllLossBackward0>)
3


100%|██████████| 258/258 [00:05<00:00, 48.17it/s]
